In [2]:
!pip install yahoo_fin


[notice] A new release of pip is available: 23.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import json
import requests
from urllib.request import urlopen
import numpy as np
import pandas as pd
from yahoo_fin import stock_info as si

In [4]:
ticker = input("Enter the ticker symbol: ").upper() 
current_price = si.get_live_price(ticker)
print(f"The current price of the stock is: {current_price:.2f}")

Enter the ticker symbol:  nvda


The current price of the stock is: 870.79


In [5]:
# Specify the file path
file_path = r'C:\Users\Code file\Intrinsic Value Calculation\Input_Intrinsic_Value_Dataset.csv'

# Read the csv file into a Pandas DataFrame
data = pd.read_csv(file_path, encoding='latin1')

# Display the DataFrame to view the data
data

,Symbol,Company Name,Market Cap,EPS Gr. Next 5Y,Beta (1Y),Shares Out,FCF,Total Debt,Total Cash,Stock Price
0,AAPL,Apple Inc.,"3,078.83B",10.38%,1.31,15.55B,99.58B,111.09B,162.10B,197.96
1,MSFT,Microsoft Corporation,"2,782.42B",15.41%,0.9,7.43B,63.23B,84.99B,143.95B,374.37
2,GOOGL,Alphabet Inc.,"1,669.14B",19.29%,1.04,12.46B,77.62B,26.33B,119.94B,133.97
3,AMZN,"Amazon.com, Inc.","1,538.11B",26.10%,1.17,10.33B,21.43B,136.99B,64.17B,148.84
4,NVDA,NVIDIA Corporation,"1,187.77B",79.09%,1.69,2.47B,17.52B,10.80B,18.28B,480.88
...,...,...,...,...,...,...,...,...,...,...
495,UAA,"Under Armour, Inc.",3.59B,10.57%,1.67,425.68M,235.83M,1.47B,1.04B,8.44
496,SMPL,The Simply Good Foods Company,3.34B,12.71%,0.7,99.60M,198.74M,272.12M,121.39M,33.48
497,SR,Spire Inc.,3.34B,7.44%,0.52,54.98M,-53.20M,4.75B,4.80M,60.7
498,CVI,"CVR Energy, Inc.",3.33B,-37.03%,1.58,100.53M,686.00M,2.19B,581.00M,33.09


In [6]:
# Remove '%' and 'B' ,'M' symbols from the specified columns
cols_to_numeric = [
    'Market Cap',
    'EPS Gr. Next 5Y',
    'Beta (1Y)',
    'Shares Out',
    'FCF',
    'Total Debt',
    'Total Cash'
]

# Function to convert values to appropriate numeric format
def convert_value(value):
    value = value.replace(',', '')
    if value == '-':
        return None  # Replace '-' with None or handle accordingly
    elif value[-1] == '%':
        return float(value[:-1])
    elif value[-1] == 'B':
        return float(value[:-1]) * 1_000_000_000
    elif value[-1] == 'M':
        return float(value[:-1]) * 1_000_000
    elif value[-1] == 'K':  
        return float(value[:-1]) * 1_000
    else:
        return float(value)

# Function to remove characters and retain original strings
def clean_specific_characters(column):
    if column.name in cols_to_numeric:
        return column.apply(convert_value)
    return column

# Apply the function to specified columns while keeping other columns as it is
data[cols_to_numeric] = data[cols_to_numeric].apply(clean_specific_characters)

# Set the display option to show full floating-point numbers
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [7]:
data

,Symbol,Company Name,Market Cap,EPS Gr. Next 5Y,Beta (1Y),Shares Out,FCF,Total Debt,Total Cash,Stock Price
0,AAPL,Apple Inc.,3078830000000.00,10.38,1.31,15550000000.00,99580000000.00,111090000000.00,162100000000.00,197.96
1,MSFT,Microsoft Corporation,2782420000000.00,15.41,0.90,7430000000.00,63230000000.00,84990000000.00,143950000000.00,374.37
2,GOOGL,Alphabet Inc.,1669140000000.00,19.29,1.04,12460000000.00,77620000000.00,26330000000.00,119940000000.00,133.97
3,AMZN,"Amazon.com, Inc.",1538110000000.00,26.10,1.17,10330000000.00,21430000000.00,136990000000.00,64170000000.00,148.84
4,NVDA,NVIDIA Corporation,1187770000000.00,79.09,1.69,2470000000.00,17520000000.00,10800000000.00,18280000000.00,480.88
...,...,...,...,...,...,...,...,...,...,...
495,UAA,"Under Armour, Inc.",3590000000.00,10.57,1.67,425680000.00,235830000.00,1470000000.00,1040000000.00,8.44
496,SMPL,The Simply Good Foods Company,3340000000.00,12.71,0.70,99600000.00,198740000.00,272120000.00,121390000.00,33.48
497,SR,Spire Inc.,3340000000.00,7.44,0.52,54980000.00,-53200000.00,4750000000.00,4800000.00,60.7
498,CVI,"CVR Energy, Inc.",3330000000.00,-37.03,1.58,100530000.00,686000000.00,2190000000.00,581000000.00,33.09


In [8]:
# Calculate the average of the 'EPS Gr. Next 5Y' column
average_eps_growth = data['EPS Gr. Next 5Y'].mean()

print(f"The average EPS growth rate for the next 5 years is: {average_eps_growth:.2f}")

The average EPS growth rate for the next 5 years is: 14.82


In [9]:
input_value = average_eps_growth
data['EPS Gr. Next 5Y'].fillna(input_value, inplace=True)

data

,Symbol,Company Name,Market Cap,EPS Gr. Next 5Y,Beta (1Y),Shares Out,FCF,Total Debt,Total Cash,Stock Price
0,AAPL,Apple Inc.,3078830000000.00,10.38,1.31,15550000000.00,99580000000.00,111090000000.00,162100000000.00,197.96
1,MSFT,Microsoft Corporation,2782420000000.00,15.41,0.90,7430000000.00,63230000000.00,84990000000.00,143950000000.00,374.37
2,GOOGL,Alphabet Inc.,1669140000000.00,19.29,1.04,12460000000.00,77620000000.00,26330000000.00,119940000000.00,133.97
3,AMZN,"Amazon.com, Inc.",1538110000000.00,26.10,1.17,10330000000.00,21430000000.00,136990000000.00,64170000000.00,148.84
4,NVDA,NVIDIA Corporation,1187770000000.00,79.09,1.69,2470000000.00,17520000000.00,10800000000.00,18280000000.00,480.88
...,...,...,...,...,...,...,...,...,...,...
495,UAA,"Under Armour, Inc.",3590000000.00,10.57,1.67,425680000.00,235830000.00,1470000000.00,1040000000.00,8.44
496,SMPL,The Simply Good Foods Company,3340000000.00,12.71,0.70,99600000.00,198740000.00,272120000.00,121390000.00,33.48
497,SR,Spire Inc.,3340000000.00,7.44,0.52,54980000.00,-53200000.00,4750000000.00,4800000.00,60.7
498,CVI,"CVR Energy, Inc.",3330000000.00,-37.03,1.58,100530000.00,686000000.00,2190000000.00,581000000.00,33.09


In [10]:
#calculating discount rate for a stock
def calculate_discount_rate(data):
    risk_free_rate = 2  # 10-year bond rate
    market_risk_premium = 5  

    # Fetching Beta values from the 'Beta(1Y)' column
    beta_values = data['Beta (1Y)']

    # Calculating the discount rate 
    discount_rate = risk_free_rate + beta_values * market_risk_premium

    return discount_rate


In [11]:
#Store discount rate column in dataset
data['Discount Rate'] = calculate_discount_rate(data)

In [12]:
data

,Symbol,Company Name,Market Cap,EPS Gr. Next 5Y,Beta (1Y),Shares Out,FCF,Total Debt,Total Cash,Stock Price,Discount Rate
0,AAPL,Apple Inc.,3078830000000.00,10.38,1.31,15550000000.00,99580000000.00,111090000000.00,162100000000.00,197.96,8.55
1,MSFT,Microsoft Corporation,2782420000000.00,15.41,0.90,7430000000.00,63230000000.00,84990000000.00,143950000000.00,374.37,6.50
2,GOOGL,Alphabet Inc.,1669140000000.00,19.29,1.04,12460000000.00,77620000000.00,26330000000.00,119940000000.00,133.97,7.20
3,AMZN,"Amazon.com, Inc.",1538110000000.00,26.10,1.17,10330000000.00,21430000000.00,136990000000.00,64170000000.00,148.84,7.85
4,NVDA,NVIDIA Corporation,1187770000000.00,79.09,1.69,2470000000.00,17520000000.00,10800000000.00,18280000000.00,480.88,10.45
...,...,...,...,...,...,...,...,...,...,...,...
495,UAA,"Under Armour, Inc.",3590000000.00,10.57,1.67,425680000.00,235830000.00,1470000000.00,1040000000.00,8.44,10.35
496,SMPL,The Simply Good Foods Company,3340000000.00,12.71,0.70,99600000.00,198740000.00,272120000.00,121390000.00,33.48,5.50
497,SR,Spire Inc.,3340000000.00,7.44,0.52,54980000.00,-53200000.00,4750000000.00,4800000.00,60.7,4.60
498,CVI,"CVR Energy, Inc.",3330000000.00,-37.03,1.58,100530000.00,686000000.00,2190000000.00,581000000.00,33.09,9.90


In [13]:
def calc_intrinsic_value(row):
    cash_flow = row['FCF']
    total_debt = row['Total Debt']
    total_cash = row['Total Cash']
    eps_growth_5Y = row['EPS Gr. Next 5Y']
    discount = row['Discount Rate']
    shs_outstanding = row['Shares Out']
    
    # Converting percentages to decimals
    eps_growth_5Y /= 100
    eps_growth_6Y_to_10Y = eps_growth_5Y / 2
    discount /= 100
    
    # Projecting future cash flows
    cf_list = []
    
    # Cash Flows Years 1 to 5
    for year in range(1, 6):
        cash_flow *= (1 + eps_growth_5Y)
        cash_flow_discount = cash_flow / ((1 + discount) ** year)
        cf_list.append(cash_flow_discount)

    # Cash Flows Years 6 to 10        
    for year in range(6, 11):
        cash_flow *= (1 + eps_growth_6Y_to_10Y)
        cash_flow_discount = cash_flow / ((1 + discount) ** year)
        cf_list.append(cash_flow_discount)

    # Intrinsic Value = Company Assets – Company Liabilities
    intrinsic_value = (sum(cf_list) - total_debt + total_cash) / shs_outstanding

    return intrinsic_value


In [14]:
# Calling the function and storing a column 'Intrinsic Value'
data['Intrinsic Value'] = data.apply(calc_intrinsic_value, axis=1)

In [15]:
# Condition to check if 'EPS Gr. Next 5Y' is negative
negative_eps_growth = data['EPS Gr. Next 5Y'] < 0

# Set 'Intrinsic Value' to 0.1 for rows where 'EPS Gr. Next 5Y' is negative
data.loc[negative_eps_growth, 'Intrinsic Value'] = 0.1

data['Intrinsic Value'] = data['Intrinsic Value'].fillna(0.1)


In [16]:
data

,Symbol,Company Name,Market Cap,EPS Gr. Next 5Y,Beta (1Y),Shares Out,FCF,Total Debt,Total Cash,Stock Price,Discount Rate,Intrinsic Value
0,AAPL,Apple Inc.,3078830000000.00,10.38,1.31,15550000000.00,99580000000.00,111090000000.00,162100000000.00,197.96,8.55,68.66
1,MSFT,Microsoft Corporation,2782420000000.00,15.41,0.90,7430000000.00,63230000000.00,84990000000.00,143950000000.00,374.37,6.50,128.21
2,GOOGL,Alphabet Inc.,1669140000000.00,19.29,1.04,12460000000.00,77620000000.00,26330000000.00,119940000000.00,133.97,7.20,107.82
3,AMZN,"Amazon.com, Inc.",1538110000000.00,26.10,1.17,10330000000.00,21430000000.00,136990000000.00,64170000000.00,148.84,7.85,36.10
4,NVDA,NVIDIA Corporation,1187770000000.00,79.09,1.69,2470000000.00,17520000000.00,10800000000.00,18280000000.00,480.88,10.45,1038.10
...,...,...,...,...,...,...,...,...,...,...,...,...
495,UAA,"Under Armour, Inc.",3590000000.00,10.57,1.67,425680000.00,235830000.00,1470000000.00,1040000000.00,8.44,10.35,4.21
496,SMPL,The Simply Good Foods Company,3340000000.00,12.71,0.70,99600000.00,198740000.00,272120000.00,121390000.00,33.48,5.50,24.93
497,SR,Spire Inc.,3340000000.00,7.44,0.52,54980000.00,-53200000.00,4750000000.00,4800000.00,60.7,4.60,-96.95
498,CVI,"CVR Energy, Inc.",3330000000.00,-37.03,1.58,100530000.00,686000000.00,2190000000.00,581000000.00,33.09,9.90,0.10


In [17]:
# Convert columns to numeric data types
data['Stock Price'] = pd.to_numeric(data['Stock Price'], errors='coerce')
data['Intrinsic Value'] = pd.to_numeric(data['Intrinsic Value'], errors='coerce')

In [18]:
# Condition to check if 'Intrinsic Value' is negative
negative_intrinsic_value = data['Intrinsic Value'] < 0

# Assign intrinsic value as half of stock value for rows where 'Intrinsic Value' is negative
data.loc[negative_intrinsic_value, 'Intrinsic Value'] = data.loc[negative_intrinsic_value, 'Stock Price'] / 2

In [19]:
def calc_percent_from_intrinsic_value(row):
    intrinsic_value = row['Intrinsic Value']
    stock_price = row['Stock Price']
    
    # Calculating the percent deviation
    percent_from_intrinsic_value = round((1 - stock_price / intrinsic_value) * 100, 2)
    
    return percent_from_intrinsic_value


In [20]:
# Calling the function and storing a column 'percent_from_instrinsic_value'
data['percent_from_instrinsic_value'] = data.apply(calc_percent_from_intrinsic_value, axis=1)

In [21]:
data

,Symbol,Company Name,Market Cap,EPS Gr. Next 5Y,Beta (1Y),Shares Out,FCF,Total Debt,Total Cash,Stock Price,Discount Rate,Intrinsic Value,percent_from_instrinsic_value
0,AAPL,Apple Inc.,3078830000000.00,10.38,1.31,15550000000.00,99580000000.00,111090000000.00,162100000000.00,197.96,8.55,68.66,-188.30
1,MSFT,Microsoft Corporation,2782420000000.00,15.41,0.90,7430000000.00,63230000000.00,84990000000.00,143950000000.00,374.37,6.50,128.21,-191.99
2,GOOGL,Alphabet Inc.,1669140000000.00,19.29,1.04,12460000000.00,77620000000.00,26330000000.00,119940000000.00,133.97,7.20,107.82,-24.25
3,AMZN,"Amazon.com, Inc.",1538110000000.00,26.10,1.17,10330000000.00,21430000000.00,136990000000.00,64170000000.00,148.84,7.85,36.10,-312.29
4,NVDA,NVIDIA Corporation,1187770000000.00,79.09,1.69,2470000000.00,17520000000.00,10800000000.00,18280000000.00,480.88,10.45,1038.10,53.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,UAA,"Under Armour, Inc.",3590000000.00,10.57,1.67,425680000.00,235830000.00,1470000000.00,1040000000.00,8.44,10.35,4.21,-100.39
496,SMPL,The Simply Good Foods Company,3340000000.00,12.71,0.70,99600000.00,198740000.00,272120000.00,121390000.00,33.48,5.50,24.93,-34.29
497,SR,Spire Inc.,3340000000.00,7.44,0.52,54980000.00,-53200000.00,4750000000.00,4800000.00,60.70,4.60,30.35,-100.00
498,CVI,"CVR Energy, Inc.",3330000000.00,-37.03,1.58,100530000.00,686000000.00,2190000000.00,581000000.00,33.09,9.90,0.10,-32990.00


In [22]:
# Save the updated DataFrame with intrinsic values to a new csv file
output_file_path = r'C:\Users\Code file\Intrinsic Value Calculation\Calculated_Intrinsic_Value_Dataset.csv'
data.to_csv(output_file_path, index=False)